In [1]:
# import os
# os.environ["R_HOME"] = "/home/ld/anaconda3/envs/r/lib/R"
import sys
sys.path.append('../')
import scDesign3

The R project used is located at D:\R-4.2.3


In [2]:
import anndata as ad
example_sce = ad.read_h5ad('test_data/VISIUM.h5ad')
example_sce.obs['test'] = ['test' for i in range(example_sce.n_obs)]
example_sce = example_sce[0:10,0:10]

In [3]:
example_sce

View of AnnData object with n_obs × n_vars = 10 × 10
    obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'spatial1', 'spatial2', 'cell_type', 'test'
    var: 'name'

construct_data interface

In [4]:
## windows
bpparam = scDesign3.get_bpparam("SnowParam",stop_on_error = False)

## linux/mac
# bpparam = test.get_bpparam("MulticoreParam",stop_on_error = False)

class: SnowParam
  bpisup: FALSE; bpnworkers: 2; bptasks: 0; bpjobname: BPJOB
  bplog: FALSE; bpthreshold: INFO; bpstopOnError: FALSE
  bpRNGseed: ; bptimeout: NA; bpprogressbar: FALSE
  bpexportglobals: TRUE; bpexportvariables: TRUE; bpforceGC: FALSE; bpfallback: TRUE
  bplogdir: NA
  bpresultdir: NA
  cluster type: SOCK



In [5]:
test = scDesign3.scDesign3(n_cores=3,parallelization="bpmapply",bpparam=bpparam)

In [6]:
const_data = test.construct_data(
    anndata=example_sce,
    corr_by="1",
    default_assay_name="counts",
    celltype='cell_type',
    spatial=['spatial1','spatial2'],
)

In [7]:
print(const_data)

$count_mat
                   Calb2 Gng4 Ttr S100a5 Nrgn Doc2g Ppp1r1b Pcbp3 Fabp7 Gpsm1
AAACAAGTATCTCCCA-1     0    0   2      0    2     1       2     1     0     0
AAACACCAATAACTGC-1    22    7   3     27    4    25       0     9    24     5
AAACAGAGCGACTCCT-1     0    1   1      0  156     0       2     4     0     0
AAACAGCTTTCAGAAG-1    16   43   2      0   30     3       0    24    17    24
AAACAGGGTCTATATT-1    19   27   2      0   15     2       1    16     7    27
AAACATGGTGAGAGGA-1    34    5   4     53    1     6       0     7    71    10
AAACATTTCCCGGATT-1     7    2   1      0   34     0       8     3     1     4
AAACCGGGTAGGTACC-1     3    2   3      0  213     4       7     4     4     1
AAACCGTTCGTCCAGG-1     2    0   4      3   16     1       5     2     2     3
AAACCTAAGCAGCCGG-1     4    0   2      0   99     1      88     4     4     8

$dat
                   cell_type spatial1 spatial2 corr_group
AAACAAGTATCTCCCA-1         2     7475     8501          1
AAACACCAA

In [8]:
# SummarizedExperiment = packages.importr('SummarizedExperiment')
# print(SummarizedExperiment.assay(test._sce,test._assay_use))
# print(SummarizedExperiment.colData(test._sce))
# print(SummarizedExperiment.rowData(test._sce))

fit_marginal interface

In [9]:
marginal = test.fit_marginal(
    mu_formula="s(spatial1,spatial2,bs='gp',k=2)",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
    trace=True,
)

In [10]:
print(marginal)

$Calb2
$Calb2$fit

Family: Negative Binomial(2.63) 
Link function: log 

Formula:
gene ~ s(spatial1, spatial2, bs = "gp", k = 2)

Estimated degrees of freedom:
2  total = 3 

REML score: 42.72744     

$Calb2$warning
$Calb2$warning[[1]]
$Calb2$warning[[1]]$function_name
[1] "gam"

$Calb2$warning[[1]]$type
[1] "warning"

$Calb2$warning[[1]]$message
[1] "simpleWarning in smooth.construct.gp.smooth.spec(object, dk$data, dk$knots): basis dimension reset to minimum possible\n"



$Calb2$time
[1] 0.1072311        NA

$Calb2$removed_cell
[1] NA


$Gng4
$Gng4$fit

Family: Negative Binomial(0.723) 
Link function: log 

Formula:
gene ~ s(spatial1, spatial2, bs = "gp", k = 2)

Estimated degrees of freedom:
2  total = 3 

REML score: 40.26233     

$Gng4$warning
$Gng4$warning[[1]]
$Gng4$warning[[1]]$function_name
[1] "gam"

$Gng4$warning[[1]]$type
[1] "warning"

$Gng4$warning[[1]]$message
[1] "simpleWarning in smooth.construct.gp.smooth.spec(object, dk$data, dk$knots): basis dimension reset to min

fit_copula interface

In [11]:
copula = test.fit_copula(
    # family_use="nb",
    n_cores=1,
    # copula = "vine",
    parallelization="mcmapply",
    # parallelization="bpmapply",
    # bpparam=bpparam,
    important_feature="auto",
)

R[write to console]: Convert Residuals to Multivariate Gaussian

R[write to console]: Converting End

R[write to console]: Copula group 1 starts



In [12]:
print(copula)

$model_aic
aic.marginal   aic.copula    aic.total 
    578.7325          Inf          Inf 

$model_bic
bic.marginal   bic.copula    bic.total 
    591.6851          Inf          Inf 

$copula_list
$copula_list$`1`
             Calb2       Gng4        Ttr      S100a5       Nrgn      Doc2g
Calb2    1.0000000  0.9828273 -0.9376622 -0.27840167  0.1244600 -0.4266383
Gng4     0.9828273  1.0000000 -0.9514097 -0.21824409  0.1089032 -0.3972863
Ttr     -0.9376622 -0.9514097  1.0000000  0.38350253 -0.2597301  0.3387435
S100a5  -0.2784017 -0.2182441  0.3835025  1.00000000 -0.2710686 -0.1144940
Nrgn     0.1244600  0.1089032 -0.2597301 -0.27106862  1.0000000 -0.1506038
Doc2g   -0.4266383 -0.3972863  0.3387435 -0.11449398 -0.1506038  1.0000000
Ppp1r1b -0.1880616 -0.2491835  0.1548442 -0.14085354  0.7945777 -0.2172680
Pcbp3    0.8355034  0.7621473 -0.7068823 -0.15241243  0.2325446 -0.2278613
Fabp7    0.1707214  0.1338153 -0.2414776  0.06540004  0.2890453  0.1903146
Gpsm1    0.6834047  0.6199111 -0.492

extract paras interface

In [13]:
paras = test.extract_para()

In [14]:
print(paras)

$mean_mat
                        Calb2       Gng4      Ttr       S100a5      Nrgn
AAACAAGTATCTCCCA-1  1.1661641  0.4462697 1.576301 2.580730e-06 104.01915
AAACACCAATAACTGC-1 22.8565810  8.5061356 3.134830 4.903413e+01  14.92508
AAACAGAGCGACTCCT-1  0.1234086  1.5024469 1.087785 5.772269e-16 209.72131
AAACAGCTTTCAGAAG-1 10.1173462 18.6730130 2.781133 2.452281e-02  24.20381
AAACAGGGTCTATATT-1 11.8839683 14.4811350 2.835565 2.694207e-01  25.69253
AAACATGGTGAGAGGA-1 48.2043693 16.3114486 3.710577 4.027848e+01   3.45063
AAACATTTCCCGGATT-1  2.8640301  0.4095792 1.860194 8.909252e-04  47.48025
AAACCGGGTAGGTACC-1  5.5054648  9.2579507 2.406118 2.051219e-02  69.68271
AAACCGTTCGTCCAGG-1  7.3546618  4.2103738 2.456332 1.628612e+00  69.45416
AAACCTAAGCAGCCGG-1  5.6088748  0.6323133 2.151165 1.267364e-01  36.10507
                         Doc2g    Ppp1r1b     Pcbp3       Fabp7      Gpsm1
AAACAAGTATCTCCCA-1  0.34684769 41.1455882  2.066016  0.30059998  1.0952989
AAACACCAATAACTGC-1  8.73636638  1.265